<a href="https://colab.research.google.com/github/Montu238/ClearView/blob/main/Smart_City_Complaint_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
#install dependencies
!pip install -q langgraph langchain langchain-google-genai


In [20]:
import os
import getpass
from langgraph.graph import StateGraph, END
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage
from typing import TypedDict


In [21]:
# Secure API Key input
GOOGLE_API_KEY = getpass.getpass("🔒 Enter your Gemini API Key: ")
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

# Initialize Gemini 1.5 Flash model using LangChain
llm = ChatGoogleGenerativeAI(
    model="models/gemini-1.5-flash-latest",
    temperature=0.2
)

🔒 Enter your Gemini API Key: ··········


In [22]:
class ComplaintState(TypedDict):
    complaint: str
    category: str
    department: str

def user_input_node(state: ComplaintState) -> ComplaintState:
    complaint = input("📝 Enter your complaint: ")
    return {"complaint": complaint, "category": "", "department": ""}


In [23]:
def classify_node(state: ComplaintState) -> ComplaintState:
    prompt = f"""
    Classify this citizen complaint into one of the following categories:
    Garbage, Pothole, Streetlight, Water, Noise, Construction.

    Complaint: "{state['complaint']}"

    Only respond with the category name.
    """
    response = llm.invoke([HumanMessage(content=prompt)])
    category = response.content.strip().split()[0]
    print(f"🧠 Gemini classified it as: {category}")
    state["category"] = category
    return state


In [24]:
CATEGORY_TO_DEPT = {
    "Garbage": "Sanitation Department",
    "Pothole": "Roads & Transport Department",
    "Streetlight": "Electricity Department",
    "Water": "Water Supply Department",
    "Noise": "Pollution Control Unit",
    "Construction": "Urban Development Authority"
}

def route_node(state: ComplaintState) -> ComplaintState:
    category = state["category"]
    dept = CATEGORY_TO_DEPT.get(category, "General Maintenance")
    print(f"📍 Routed to: {dept}")
    state["department"] = dept
    return state


In [25]:
builder = StateGraph(ComplaintState)

# Add nodes
builder.add_node("user_input", user_input_node)
builder.add_node("classify", classify_node)
builder.add_node("route", route_node)

# Optional output node
def user_output_node(state: ComplaintState):
    print("\n✅ Complaint Processed Successfully!")
    print(f"📝 Complaint     : {state['complaint']}")
    print(f"📂 Category      : {state['category']}")
    print(f"🏢 Department    : {state['department']}")
    return state

builder.add_node("user_output", user_output_node)

# Set entry and transitions
builder.set_entry_point("user_input")
builder.add_edge("user_input", "classify")
builder.add_edge("classify", "route")
builder.add_edge("route", "user_output")
builder.add_edge("user_output", END)


In [26]:
graph = builder.compile()

initial_state = {
    "complaint": "",
    "category": "",
    "department": ""
}

graph.invoke(initial_state)


📝 Enter your complaint: road from city bus stand to mg college is very poor and needs maintenance
🧠 Gemini classified it as: Pothole
📍 Routed to: Roads & Transport Department

✅ Complaint Processed Successfully!
📝 Complaint     : road from city bus stand to mg college is very poor and needs maintenance
📂 Category      : Pothole
🏢 Department    : Roads & Transport Department


{'complaint': 'road from city bus stand to mg college is very poor and needs maintenance',
 'category': 'Pothole',
 'department': 'Roads & Transport Department'}